In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

In [2]:
dbT= pd.read_excel("../DataFiles/Traffic to Contract History.xlsx")
dbCH= pd.read_excel("../DataFiles/Contract History.xlsx")
dbCO= pd.read_excel("../DataFiles/Contracts.xlsx")
SU = pd.read_excel("../DataFiles/Survey Units.xlsx")
SDD = pd.read_excel("../DataFiles/Survey Distress Details.xlsx")

In [3]:
dbT.head()

,Traffic Year,AADT,Single Trucks,Double Trucks,Train Trucks,Peak Hour Percentage,Peak Hour Directional Split,Traffic Year ESALs _One Direction_,Percent Trucks,Contract History ID
0,2013,31462,843,280,22,7.86,50.99,119738,3.64,7fbae59b-6ee6-4419-acb0-1db37aacd5f4
1,2013,23939,642,213,17,7.87,53.53,91031,3.64,39483404-7908-4e89-ade2-bec2391a60b6
2,2013,23939,642,213,17,7.87,53.53,91031,3.64,520f0233-9842-4d00-b92e-a10522a50102
3,2013,19593,525,174,14,8.00,52.73,74348,3.64,f042adcf-fe53-4442-be3c-66080c43e0e7
4,2013,20944,561,186,15,8.02,53.16,79442,3.64,5a9fd6df-7564-4251-a8ee-69f18e1f6e70


In [4]:
#dbT = dbT.drop(['Single Trucks', 'Double Trucks', 'Peak Hour Percentage', 'Peak Hour Directional Split'], axis =1)
dbT = dbT[['Traffic Year', 'Traffic Year ESALs _One Direction_', 'Contract History ID']]
dbT.columns = ['Traffic Year', 'ESALs', 'Contract History ID']

In [5]:
dbCH = dbCH[['SR','Begin ARM', 'End ARM', 'Region', 'ID', 'Construction Year', 'Major Rehab Year', 'Segment Lane Miles', 
            'ESALs Since Major Rehab', 'Total Thickness', 'Total Surface Thickness', 'Listed Surface Type', 'Pavement Type',
            'Construction Pavement Type', 'Major Rehab Type', 'Prior Major Rehab Year', 'Prior Major Rehab Type']]

In [6]:
T_CH = pd.merge(dbT, dbCH, how = 'left',left_on = ['Contract History ID'], right_on = ['ID'])

In [7]:
print len(T_CH)
print len(dbCH)
print len(dbT) == len(T_CH)

21892
28588
True


In [8]:
dbCO.columns

Index([                         u'SR',                         u'RRT',
                               u'RRQ',                   u'Begin ARM',
                           u'End ARM',                  u'Begin SRMP',
                          u'End SRMP',      u'Direction To Inventory',
                                u'ID',             u'Contract Number',
             u'Construction End Date', u'Substantial Completion Date',
                u'Contract Type Code',   u'Contract Type Description',
                    u'Exception Code',       u'Exception Description',
                 u'Surface Type Code',    u'Surface Type Description',
                 u'Surface Thickness',        u'Treat Base Type Code',
       u'Treat Base Type Description',        u'Treat Base Thickness',
             u'UTreat Base Thickness',             u'Sequence Number'],
      dtype='object')

In [9]:
T_CH.columns

Index([              u'Traffic Year',                      u'ESALs',
              u'Contract History ID',                         u'SR',
                        u'Begin ARM',                    u'End ARM',
                           u'Region',                         u'ID',
                u'Construction Year',           u'Major Rehab Year',
               u'Segment Lane Miles',    u'ESALs Since Major Rehab',
                  u'Total Thickness',    u'Total Surface Thickness',
              u'Listed Surface Type',              u'Pavement Type',
       u'Construction Pavement Type',           u'Major Rehab Type',
           u'Prior Major Rehab Year',     u'Prior Major Rehab Type'],
      dtype='object')

In [10]:
dbC = dbCO[['SR', 'Begin ARM', 'End ARM', 'ID','Construction End Date','Substantial Completion Date','Contract Type Description', 'Surface Type Code', 'Surface Type Description', 'Surface Thickness',
          'Treat Base Type Code', 'Treat Base Type Description', 'Treat Base Thickness', 'UTreat Base Thickness']]

In [11]:
# ConDate = dbC['Construction End Date']#.str.split("-")
# dbC['Year'] = pd.DatetimeIndex(dbC['Construction End Date']).year
# str(dbC.loc[4,'Construction End Date']).split('-')[0]
# dbC= dbC.drop("Year", axis = 1)
# test = dbCO.loc[:100, ['Construction End Date', 'Substantial Completion Date']]
# res = test['Construction End Date'].apply(getYear)
# test['Year'] = res

In [12]:
dbC.loc[:,'Year'] = 0
def getYear(year):
    return int(str(year).split('-')[0])

/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
for i in np.arange(len(dbC)):
    dbC.loc[i, 'Year'] = int(str(dbC.loc[i,'Construction End Date']).split('-')[0])

In [14]:
# dbC.tail(10)

I have split up the construction date column and created a 'Year' column which I will use to select the actions which were performed before year 2000. 

In [14]:
dbC2 = dbC[dbC.loc[:, 'Year'] <= 2013]

In [16]:
print len(dbC)
print len(dbC2)

54196
51665


In [17]:
ConType = np.unique(np.array(dbC.loc[:, 'Contract Type Description']))
print ConType

[nan u'2ND STAGE CONSTR' u'2ND STAGE RECONSTR' u'BRIDGE'
 u'CI, CO, FED ROAD PROJECT' u'FIRST STAGE CONSTR' u'FIRST STAGE RECONSTR'
 u'FIRST STAGE RESURFACING' u'NEW CONSTR OR ALIGN CHG' u'PLANING'
 u'RECONSTR EXISTING ALIGN' u'RESURFACE EXISTING RDWY' u'TUNNEL']


/home/guerrero/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:198: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [18]:
# ConType = np.delete(ConType, [0,3,4, 9, 12])
SurfType = [ConType[1], ConType[2], ConType[5], ConType[6], ConType[7],  ConType[8], ConType[10],  ConType[11]]
print SurfType

[u'2ND STAGE CONSTR', u'2ND STAGE RECONSTR', u'FIRST STAGE CONSTR', u'FIRST STAGE RECONSTR', u'FIRST STAGE RESURFACING', u'NEW CONSTR OR ALIGN CHG', u'RECONSTR EXISTING ALIGN', u'RESURFACE EXISTING RDWY']


In [19]:
dbCs = dbC2[dbC2.loc[:,'Contract Type Description'].isin(SurfType)]

In [20]:
len(dbCs)

42681

In [29]:
# print len(dbCs[dbCs.loc[:, 'UTreat Base Thickness']> 0.0])
# print len(dbCs[dbCs.loc[:, 'Treat Base Thickness']> 0.0])
# print len(dbCs)

From the above summary, it can be observed that the number of segments where the treated or non-treated base layer thickness was recorded is less than 0.2 percent of the total segments in the dataset. The base thickness variable will therefore be removed for the models using records of year less than 2000. It should be noted, however, that for records of year >= 2000, there is a significant number of records with treated and untreated base-layer thicknesses so included of these variables into the models can be warranted.

In [21]:
db = dbCs[['SR', 'Begin ARM', 'End ARM', 'ID','Year','Contract Type Description', 'Surface Type Code', 'Surface Type Description', 'Surface Thickness']]

In [22]:
print len(dbCH), len(db)

28588 42681


In [23]:
C_CH = pd.merge(db, dbCH, how = 'left',left_on = ['SR', 'Begin ARM', 'End ARM'], right_on = ['SR', 'Begin ARM', 'End ARM'])

In [24]:
# Missing = C_CH[C_CH['Construction Year'].apply(np.isnan)]
# CCHs = C_CH[~C_CH.index.isin(Missing.index)]
CCHs = C_CH[~C_CH['Construction Year'].apply(np.isnan)]
print len(CCHs), len(C_CH)

41165 67500


In [27]:
CCHs.columns

Index([                        u'SR',                  u'Begin ARM',
                          u'End ARM',                       u'ID_x',
                             u'Year',  u'Contract Type Description',
                u'Surface Type Code',   u'Surface Type Description',
                u'Surface Thickness',                     u'Region',
                             u'ID_y',          u'Construction Year',
                 u'Major Rehab Year',         u'Segment Lane Miles',
          u'ESALs Since Major Rehab',            u'Total Thickness',
          u'Total Surface Thickness',        u'Listed Surface Type',
                    u'Pavement Type', u'Construction Pavement Type',
                 u'Major Rehab Type',     u'Prior Major Rehab Year',
           u'Prior Major Rehab Type'],
      dtype='object')

The following subselection contains only those segments with values in both the Contracts database and that in Contracts-History database.

In [28]:
CCHs.index = np.arange(len(CCHs))

In [30]:
MatType = np.unique(np.array(dbCs.loc[:, 'Surface Type Code']))

In [31]:
MTdic = {}
for mt in MatType[1:]:
    MTdic[mt] = len(dbCs[dbCs.loc[:, 'Surface Type Code']==mt])

In [32]:
uniqMT = []
for mt in MTdic.keys():
    if MTdic[mt] > 1000:
        print mt, MTdic[mt]
        uniqMT.append(mt)

BA 10887
G2 1300
B2 2394
PA 2251
SA 3864
TA 3725
AA 1458


In [33]:
uniqMT

[u'BA', u'G2', u'B2', u'PA', u'SA', u'TA', u'AA']

In [34]:
# # CCHs[~CCHs.loc[:,'Surface Type Code'].isin(uniqMT)].loc[:,'Surface Type Code']
# for i in CCHs.index:
#     if CCHs.loc[i,'Surface Type Code'] not in uniqMT:
#         CCHs.loc[i,'Surface Type Code'] = 'Other'  

I had categorized the surface type previously but later realized that the final dataset might not contain the same proportion of variables, given that I am going to consider only asphalt pavements. Categorization of this nature will be among the final steps of the data cleaning process..

Just concluded categorizing the Surface Type Code accordingly. Next step is to link the CCHs (Contract and Contract-History) dataframe to the Survey_Units dataframe using the linker dataframe uploaded below..

In [36]:
# SUL = pd.read_excel("../DataFiles/Survey Units-Contract History Units.xlsx")
# SUL = SUL[[u'Survey Unit ID', u'Contract History ID', u'SR', u'Begin ARM',u'End ARM', u'Lane Miles']]
# SUL.columns = [u'Survey Unit ID', u'Contract History ID', u'SR_SU', u'Begin ARM_SU',u'End ARM_SU', u'Lane Miles']
# CCH = pd.merge(CCHs, SUL, how = 'left',left_on = ['SR', 'Begin ARM', 'End ARM'], 
#                right_on = ['SR_SU', 'Begin ARM_SU', 'End ARM_SU'])
# Merged = CCH[~CCH['Lane Miles'].apply(np.isnan)]

The linker dataframe is limited in the segment pairs it links so from now onwards, its best to stick to the SR and route-mileage as the unique indicators across all the datasets.

In [37]:
len(CCHs[CCHs.Year == CCHs.loc[:, 'Major Rehab Year']])

8425

In [40]:
pvmtType = np.unique(np.array(CCHs[['Pavement Type']]))
print pvmtType

[nan u'ACP' u'ACP over BST (Mixed)' u'BST' u'BST over ACP (Mixed)' u'CSTS'
 u'Composite' u'DBR' u'Grinding PCCP' u'PCCP']


In [45]:
# selected pavement type
spt = [pvmtType[1], pvmtType[2], pvmtType[3], pvmtType[4],  pvmtType[6]]
print spt

[u'ACP', u'ACP over BST (Mixed)', u'BST', u'BST over ACP (Mixed)', u'Composite']


In [46]:
CCHa = CCHs[CCHs.loc[:,'Pavement Type'].isin(spt)]

In [48]:
print len(CCHa)
print len(CCHa[CCHa.Year == CCHa.loc[:, 'Major Rehab Year']])

36943
7152


# Merging survey units (SU) and Survey Distress Details (SDD)

In [64]:
SU.columns

Index([                      u'SR',                      u'RRT',
                            u'RRQ',                u'Begin ARM',
                        u'End ARM',               u'Begin SRMP',
                       u'End SRMP',   u'Direction To Inventory',
                    u'Original ID',                 u'Due Year',
               u'Due Year Trigger',                u'Is Bridge',
                 u'Is Near Tunnel',      u'Is Near RR Crossing',
              u'Lane Surface Type',               u'Lane Miles',
                         u'Region',                      u'NHS',
       u'Federal Functional Class',    u'Avg Construction Date',
           u'Avg Major Rehab Date',                   u'Int ID'],
      dtype='object')

In [65]:
su = SU[[u'SR', u'Begin ARM', u'End ARM', u'Is Bridge', u'Lane Surface Type', u'Lane Miles', u'Region', 
         u'Federal Functional Class', u'Int ID']]

In [66]:
su.head()

,SR,Begin ARM,End ARM,Is Bridge,Lane Surface Type,Lane Miles,Region,Federal Functional Class,Int ID
0,2,312.04,312.14,False,BST,0.20,Eastern,Rural Other Freeway/Expressway,1
1,2,171.15,171.25,False,BST,0.20,North Central,Rural Other Principal Arterial,2
2,2,269.53,269.63,False,ACP,0.20,Eastern,Urban Other Principal Arterial,3
3,2,22.12,22.23,True,ACP,0.22,Northwest,Urban Other Principal Arterial,4
4,2,115.81,115.91,False,ACP,0.20,North Central,Urban Other Freeway/Expressway,5


In [63]:
# SDD.columns

In [61]:
sdd = SDD[[u'Year',u'Low Alligator Cracking',u'Medium Alligator Cracking',u'High Alligator Cracking',
           u'Low Flexible Longitudinal Cracking', u'Medium Flexible Longitudinal Cracking',
           u'High Flexible Longitudinal Cracking', u'Int ID']]
sdd.columns = [u'sYear',u'Low Alligator Cracking',u'Medium Alligator Cracking',u'High Alligator Cracking',
           u'Low Flexible Longitudinal Cracking', u'Medium Flexible Longitudinal Cracking',
           u'High Flexible Longitudinal Cracking', u'IntID']

In [62]:
sdd.head()

,sYear,Low Alligator Cracking,Medium Alligator Cracking,High Alligator Cracking,Low Flexible Longitudinal Cracking,Medium Flexible Longitudinal Cracking,High Flexible Longitudinal Cracking,IntID
0,2003,NaN,NaN,NaN,2,NaN,NaN,54967
1,2003,2,NaN,NaN,16,NaN,NaN,54965
2,2003,19,NaN,NaN,19,NaN,NaN,54963
3,2003,NaN,NaN,NaN,10,NaN,NaN,54961
4,2003,NaN,NaN,NaN,10,NaN,NaN,54962


In [58]:
len(np.unique(np.array(sdd[['Construction']])))

1036027

Code below merges the two datasets, su and sdd.. (survey units, and survey distress details)

In [70]:
SUDD = pd.merge(sdd, su, how = 'left',left_on = ['IntID'],
               right_on = ['Int ID'])
sudd = SUDD.dropna(axis=0, how='any', thresh=None, subset= [u'SR', u'Begin ARM', u'End ARM', u'Is Bridge', u'Lane Surface Type', u'Lane Miles', u'Region', 
         u'Federal Functional Class'], inplace=False)

In [71]:
print len(sudd)
print len(SUDD)
print len(sudd) == len(sdd)

1048575
1048575
True


# Code below merges the two datasets

In [83]:
CCHa.columns = [u'SR', u'Begin ARM', u'End ARM', u'ID_x', u'cYear',  u'Contract Type Description',
                u'Surface Type Code', u'Surface Type Description', u'Surface Thickness',u'Region', u'ID_y', 
                u'Construction Year', u'Major Rehab Year', u'Segment Lane Miles', u'ESALs Since Major Rehab', 'Total Thickness',
                u'Total Surface Thickness', u'Listed Surface Type', u'Pavement Type', u'Construction Pavement Type',
                u'Major Rehab Type', u'Prior Major Rehab Year', u'Prior Major Rehab Type']

In [93]:
CCHp = CCHa.drop_duplicates(subset = ['SR', 'Begin ARM', 'End ARM', 'cYear'])

In [94]:
SUDDCCH = pd.merge(sudd, CCHp, how = 'left',left_on = ['SR', 'Begin ARM', 'End ARM', 'Region'],
               right_on = ['SR', 'Begin ARM', 'End ARM', 'Region'])
suddcch = SUDDCCH.dropna(axis=0, how='any', thresh=None, subset= [ u'Surface Thickness','cYear'], inplace=False)

In [98]:
print len(suddcch), len(SUDDCCH), len(CCHp), len(CCHa)
print len(suddcch) == len(sudd)
tdf = suddcch[suddcch.cYear == suddcch.loc[:, 'Major Rehab Year']]
print len(np.unique(np.array(tdf.IntID)))

39782 1068044 16316 36943
False
650


In [91]:
sudd.head()

,sYear,Low Alligator Cracking,Medium Alligator Cracking,High Alligator Cracking,Low Flexible Longitudinal Cracking,Medium Flexible Longitudinal Cracking,High Flexible Longitudinal Cracking,IntID,SR,Begin ARM,End ARM,Is Bridge,Lane Surface Type,Lane Miles,Region,Federal Functional Class,Int ID
0,2003,NaN,NaN,NaN,2,NaN,NaN,54967,101,0.00,0.10,False,ACP,0.20,Southwest,Rural Minor Arterial,54967
1,2003,2,NaN,NaN,16,NaN,NaN,54965,101,0.10,0.20,False,ACP,0.20,Southwest,Rural Minor Arterial,54965
2,2003,19,NaN,NaN,19,NaN,NaN,54963,101,0.20,0.29,False,ACP,0.18,Southwest,Rural Minor Arterial,54963
3,2003,NaN,NaN,NaN,10,NaN,NaN,54961,101,0.29,0.39,False,ACP,0.20,Southwest,Rural Minor Arterial,54961
4,2003,NaN,NaN,NaN,10,NaN,NaN,54962,101,0.39,0.49,False,ACP,0.20,Southwest,Rural Minor Arterial,54962


In [99]:
CCHp.head()

,SR,Begin ARM,End ARM,ID_x,cYear,Contract Type Description,Surface Type Code,Surface Type Description,Surface Thickness,Region,...,Segment Lane Miles,ESALs Since Major Rehab,Total Thickness,Total Surface Thickness,Listed Surface Type,Pavement Type,Construction Pavement Type,Major Rehab Type,Prior Major Rehab Year,Prior Major Rehab Type
0,2,0.00,0.13,206e5b45-4e3b-4e3f-9a86-d284970ae774,1994,RECONSTR EXISTING ALIGN,BA,ACP CLASS B AR4000W,0.25,Eastern,...,0.39,1337098,1.90,1.40,ACP,Composite,PCCP,ACP,1994,ACP
2,2,0.10,0.13,b41195f8-d576-4d66-ac74-6e30f296e19f,2002,RESURFACE EXISTING RDWY,B5,ACP CLASS A PG64-22,0.15,Northwest,...,0.06,592521,1.15,0.40,ACP,ACP,ACP,ACP,1994,ACP
3,2,0.13,0.14,e4f50ef8-2cb1-41c7-8ef8-e33553ca5359,2002,RESURFACE EXISTING RDWY,B5,ACP CLASS A PG64-22,0.15,Northwest,...,0.02,627668,0.15,0.15,ACP,Composite,PCCP,ACP,1968,PCCP
4,2,0.13,0.16,bec16537-c2dc-404b-be51-0139edccbd7f,1986,NEW CONSTR OR ALIGN CHG,BA,ACP CLASS B AR4000W,0.00,Northwest,...,0.06,585006,0.15,0.15,ACP,ACP,ACP,ACP,1986,ACP
5,2,0.13,0.16,a7c96460-896f-47e1-a547-4c47c00fe818,2005,RESURFACE EXISTING RDWY,AA,ACP CLASS A AR4000W,0.15,Northwest,...,0.06,585006,0.15,0.15,ACP,ACP,ACP,ACP,1986,ACP


In [103]:
su.head()

,SR,Begin ARM,End ARM,Is Bridge,Lane Surface Type,Lane Miles,Region,Federal Functional Class,Int ID
0,2,312.04,312.14,False,BST,0.20,Eastern,Rural Other Freeway/Expressway,1
1,2,171.15,171.25,False,BST,0.20,North Central,Rural Other Principal Arterial,2
2,2,269.53,269.63,False,ACP,0.20,Eastern,Urban Other Principal Arterial,3
3,2,22.12,22.23,True,ACP,0.22,Northwest,Urban Other Principal Arterial,4
4,2,115.81,115.91,False,ACP,0.20,North Central,Urban Other Freeway/Expressway,5


In [123]:
Merged = SUCCH[~SUCCH['Lane Miles'].apply(np.isnan)]

In [124]:
print len(Merged)
print len(Merged[Merged.loc[:,'Year']>1980])
print len(np.unique(np.array(Merged[Merged.loc[:,'Year']>1980].loc[:, 'Int ID'])))

7065
6249
1620


In [199]:
len(np.unique(np.array(Merged.loc[:, [u'SR', u'Begin ARM', u'End ARM']])))

2357

Adding the Survey Details dataset including the cracking variables

In [175]:
# SDD.columns

In [158]:
sdd = SDD[[u'Year', u'Low Alligator Cracking', u'Medium Alligator Cracking',u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking',u'Medium Flexible Longitudinal Cracking', u'High Flexible Longitudinal Cracking',u'Sealed Alligator',
           u'Sealed Longitudinal', u'Bridge', u'Int ID']]

In [159]:
sdd.columns = [u'YearC', u'Low Alligator Cracking', u'Medium Alligator Cracking',u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking',u'Medium Flexible Longitudinal Cracking', u'High Flexible Longitudinal Cracking',u'Sealed Alligator',
           u'Sealed Longitudinal', u'Bridge', u'Int ID']

# Merging Cracking Variable dataset to Segment Properties DB

In [160]:
#SCV segment and cracking variables
SCV = pd.merge(sdd, Merged, how = 'left',left_on = ['Int ID'],
               right_on = ['Int ID'])

In [161]:
len(SCV)

1106846

In [217]:
scv = SCV[~SCV[['Lane Miles']].apply(np.isnan)]

In [218]:
len(scv)

1106846

In [200]:
len(np.unique(np.array(scv.loc[:, [u'SR', u'Begin ARM', u'End ARM']])))

1402

In [137]:
# scv.to_excel("../DataFiles/seg_cracking_vars2010.xlsx")

In [165]:
#sc is the dataset with selected vars necessary for formulating the crack initiation model
sc = scv[[u'YearC', u'Low Alligator Cracking', u'Medium Alligator Cracking', u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking', u'Medium Flexible Longitudinal Cracking', u'High Flexible Longitudinal Cracking',u'Sealed Alligator',
          u'Sealed Longitudinal', u'Int ID', u'SR', u'Begin ARM', u'End ARM', u'Year', u'Contract Type Description',
          u'Surface Type Code', u'Surface Type Description', u'Surface Thickness', u'Region', u'ID_y', u'Construction Year',
          u'Major Rehab Year', u'Segment Lane Miles', u'ESALs Since Major Rehab', u'Total Thickness', u'Total Surface Thickness',
          u'Pavement Type', u'Construction Pavement Type', u'Major Rehab Type',u'Prior Major Rehab Year',
          u'Prior Major Rehab Type', u'Original ID', u'Is Bridge', u'Lane Surface Type', u'Lane Miles',
                    u'Federal Functional Class',  u'Avg Construction Date',   u'Avg Major Rehab Date']]

In [172]:
# sc.head()

In [167]:
sc.loc[:,'Alli'] = 'nan'
sc.loc[:,'CatAlli'] = 'nan'
sc.loc[:,"Long"] = 'nan'
sc.loc[:,"CatLong"] = 'nan'

In [168]:
for i in sc.index:
    if ~np.isnan(sc.loc[i,'Low Alligator Cracking']):
        sc.loc[i,'Alli'] = sc.loc[i,'Low Alligator Cracking'] 
        sc.loc[i,'CatAlli'] = 'L'
    elif ~np.isnan(sc.loc[i,'Medium Alligator Cracking']):
        sc.loc[i,'Alli'] = sc.loc[i,'Medium Alligator Cracking'] 
        sc.loc[i,'CatAlli'] = 'M'
    else:
        sc.loc[i,'Alli'] = sc.loc[i,'High Alligator Cracking'] 
        sc.loc[i,'CatAlli'] = 'H'
        
    if ~np.isnan(sc.loc[i,'Low Flexible Longitudinal Cracking']):
        sc.loc[i,'Long'] = sc.loc[i,'Low Flexible Longitudinal Cracking'] 
        sc.loc[i,'CatLong'] = 'L'
    elif ~np.isnan(sc.loc[i,'Medium Flexible Longitudinal Cracking']):
        sc.loc[i,'Long'] = sc.loc[i,'Medium Flexible Longitudinal Cracking'] 
        sc.loc[i,'CatLong'] = 'M'
    else:
        sc.loc[i,'Long'] = sc.loc[i,'High Flexible Longitudinal Cracking'] 
        sc.loc[i,'CatLong'] = 'H'

In [278]:
test = sc.loc[1:1000, ['Alli', 'Long']]

In [279]:
test.index = np.arange(len(test))

In [295]:
test2 = test.loc[1:10, ['Alli', 'Long']]
print test2.head()

  Alli Long
1   10   84
2   21   10
3   21    5
4  NaN    5
5  NaN  NaN


In [325]:
import math

In [339]:
def addcr(row):
    a= row['Alli']
    b= row['Long']
#     return not (math.isnan(a) & math.isnan(b))
    if not (math.isnan(a) & math.isnan(b)):
        return a+b
    else:
        return 999

In [337]:
a = 86
b = 56
print (isinstance(a, int) & isinstance(b, int))

True


In [340]:
test2.apply(addcr, axis=1)

1      94
2      31
3      26
4     NaN
5     999
6     999
7     999
8     999
9     999
10    999
dtype: float64

In [ ]:
def catme(p1,p2,p3,p4,p5,p6):
    if isinstance(p1, int):
        a = p1
        at = 'L'
    elif isinstance(p2, int):
        a = p2
        at = 'M'
    elif isinstance(p3, int):
        a = p3
        at = 'H'
    else:
        a = 'nan'
        at = 'nan'
        
    if isinstance(p4, int):
        l = p4
        lt = 'L'
    elif isinstance(p5, int):
        l = p5
        lt = 'M'
    elif isinstance(p6, int):
        l = p6
        lt = 'H'
    else:
        l = 'nan'
        lt = 'nan'
    return a, at, l, lt

In [170]:
df = sc[[u'YearC',u'Sealed Alligator', u'Sealed Longitudinal', u'Int ID', u'SR', u'Begin ARM', u'End ARM', u'Year', 
         u'Contract Type Description', u'Surface Type Code', u'Surface Type Description', u'Surface Thickness', 
         u'Region', u'ID_y', u'Construction Year', u'Major Rehab Year', u'Segment Lane Miles', u'ESALs Since Major Rehab',
         u'Total Thickness', u'Total Surface Thickness', u'Pavement Type', u'Construction Pavement Type', 
         u'Major Rehab Type',u'Prior Major Rehab Year', u'Prior Major Rehab Type', u'Original ID', 
         u'Is Bridge', u'Lane Surface Type', u'Lane Miles', u'Federal Functional Class',  u'Avg Construction Date', 
         u'Avg Major Rehab Date', 'Alli', 'CatAlli', 'Long', 'CatLong']]

In [202]:
# df.head()

# Merging dataframe with traffic data

In [176]:
Traffic = pd.read_excel("../DataFiles/Traffic.xlsx")

In [182]:
Traffic.columns

Index([                                u'SR',
                                      u'RRT',
                                      u'RRQ',
                                u'Begin ARM',
                                  u'End ARM',
                               u'Begin SRMP',
                                 u'End SRMP',
                   u'Direction To Inventory',
                             u'Traffic Year',
                                     u'AADT',
                            u'Single Trucks',
                            u'Double Trucks',
                             u'Train Trucks',
                     u'Peak Hour Percentage',
              u'Peak Hour Directional Split',
       u'Traffic Year ESALs _One Direction_',
                           u'Percent Trucks'],
      dtype='object')

In [245]:
len(np.unique(np.array(Traffic.loc[:, [u'SR',u'Begin ARM',u'End ARM']])))

4784

In [185]:
Tc = Traffic[[u'SR', u'Begin ARM', u'End ARM', u'Direction To Inventory', u'Traffic Year', u'AADT', 
       u'Traffic Year ESALs _One Direction_']]

In [186]:
Tc.columns = [u'SR', u'Begin ARM', u'End ARM', u'Direction To Inventory', u'Traffic Year', u'AADT', 
       u'ESALS']

In [191]:
SCT = pd.merge(df, Tc, how = 'left',left_on = ['SR', u'Begin ARM', u'End ARM'],
               right_on = ['SR', u'Begin ARM', u'End ARM'])

In [227]:
#Trying out the use of dropna in pandas. Foud out it works to using apply(np.isnan) on the respective column.
# test = SCT[[u'Direction To Inventory', u'Traffic Year', u'AADT', u'ESALS']]
# len(test.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False))
# test2 = SCT.dropna(axis=0, how='any', thresh=None, 
#                    subset=[u'Direction To Inventory', u'Traffic Year', u'AADT', u'ESALS'], inplace=False)
# len(test2)

In [225]:
sct = SCT[~SCT['AADT'].apply(np.isnan)]

In [230]:
print len(sct)
print len(SCT)

3490
80734


In [197]:
len(np.unique(np.array(Traffic.loc[:, [u'SR', u'Begin ARM', u'End ARM']])))

4784

In [248]:
dbT.head()

,Traffic Year,ESALs,Contract History ID
0,2013,119738,7fbae59b-6ee6-4419-acb0-1db37aacd5f4
1,2013,91031,39483404-7908-4e89-ade2-bec2391a60b6
2,2013,91031,520f0233-9842-4d00-b92e-a10522a50102
3,2013,74348,f042adcf-fe53-4442-be3c-66080c43e0e7
4,2013,79442,5a9fd6df-7564-4251-a8ee-69f18e1f6e70


In [249]:
len(np.unique(np.array(dbT.loc[:, [u'Contract History ID']])))

21892

# Merging the Traffic_to_Contract_History database to df

In [252]:
df2 = pd.merge(df, dbT, how = 'left',left_on = ['ID_y'], right_on = ['Contract History ID'])

In [253]:
len(np.unique(np.array(df2.loc[:, [u'Contract History ID']])))

914

In [254]:
dft = df2.dropna(axis=0, how='any', thresh=None, 
                   subset=[u'Traffic Year', u'ESALs'], inplace=False)

In [257]:
def getST(index):
    return dft.loc[index, 'Surface Type Code']

In [258]:
dfst = dft.drop_duplicates(['Contract History ID'])

In [263]:
len(dfst[dfst.loc[:,'Surface Type Code']=='Other'])

561

In [262]:
for mt in uniqMT:
    print mt, len(dfst[dfst.loc[:,'Surface Type Code']== mt])

BA 227
B2 68
SA 12
TA 8
AA 37


In [255]:
print len(df2)
print len(dft)

77650
64452


In [256]:
dft.head()

,YearC,Sealed Alligator,Sealed Longitudinal,Int ID,SR,Begin ARM,End ARM,Year,Contract Type Description,Surface Type Code,...,Federal Functional Class,Avg Construction Date,Avg Major Rehab Date,Alli,CatAlli,Long,CatLong,Traffic Year,ESALs,Contract History ID
0,2003,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,Rural Minor Arterial,1944-01-01,1995-07-31,21,L,10,L,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d
1,2004,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,Rural Minor Arterial,1944-01-01,1995-07-31,10,L,84,L,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d
2,2005,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,Rural Minor Arterial,1944-01-01,1995-07-31,21,L,10,L,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d
3,2006,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,Rural Minor Arterial,1944-01-01,1995-07-31,21,L,5,L,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d
4,2007,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,Rural Minor Arterial,1944-01-01,1995-07-31,NaN,H,5,L,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d


# Given that you know which datasets are now required, repeat the process and do the Surface-material type categorization at the end.

In [ ]:
def getAlli(row):
    p1 = row[u'Low Alligator Cracking']
    p2 = row[u'Medium Alligator Cracking']
    p3 = row[u'High Alligator Cracking']
    
    if not math.isnan(p1):
        a = p1
    elif not math.isnan(p2):
        a = p2
    elif not math.isnan(p3):
        a = p3
        at = 'H'
    else:
        a = 'nan'
    return a

In [ ]:
def getCatAlli(row):
    p1 = row[u'Low Alligator Cracking']
    p2 = row[u'Medium Alligator Cracking']
    p3 = row[u'High Alligator Cracking']
    
    if not math.isnan(p1):
        at = 'L'
    elif not math.isnan(p2):
        at = 'M'
    elif not math.isnan(p3):
        at = 'H'
    else:
        at = 'nan'
    return at

In [ ]:
def getCatLong(row):
    p4 = row[u'Low Flexible Longitudinal Cracking']
    p5 = row[u'Medium Flexible Longitudinal Cracking']
    p6 = row[u'High Flexible Longitudinal Cracking'] 
        
    if not math.isnan(p4):
        lt = 'L'
    elif not math.isnan(p5):
        lt = 'M'
    elif not math.isnan(p6):
        lt = 'H'
    else:
        lt = 'nan'
    return lt

In [ ]:
def getLong(row):
    p4 = row[u'Low Flexible Longitudinal Cracking']
    p5 = row[u'Medium Flexible Longitudinal Cracking']
    p6 = row[u'High Flexible Longitudinal Cracking'] 
        
    if not math.isnan(p4):
        l = p4
    elif not math.isnan(p5):
        l = p5
    elif not math.isnan(p6):
        l = p6
    else:
        l = 'nan'
    return l